<a href="https://colab.research.google.com/github/Lutris98/PersonalProject/blob/main/2.Analytics/21.1Q_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

투자 개요<br>
미래전망을보고 종목선정<br>
4개정도의 종목선정후 포트폴리오구성<br>
구성후엔 매수, 매도 시점 확인 

#1.투자개요

투자할 종목은 <br>
다다 넥서스(DADA)<br>
마이크로소프트(MSFT)<br>
화이자(PFE)<br>

#2.포트폴리오 구성

In [1]:
!pip install yfinance
!pip install mplfinance

     |████████████████████████████████| 5.5MB 6.3MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=2b5f6b89fea44b879af87ec40b96a834eb83904b0cdd1ae0a04fed326754abb6
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 61kB 3.2MB/s 


In [ ]:
from pandas_datareader import data as pdr #datareader가 필요함
import yfinance as yf #해외주식 한정 정확
import datetime
yf.pdr_override()
df=pdr.get_data_yahoo('DADA')
df=df[['Open','High','Low','Close','Volume']]
df=df.iloc[-91:-1] #실제 판단은 한 분기단위로 
df

In [ ]:
daily_ret=df.pct_change() #dpc가 아예 판다스 내장함수로 구현되어있음
daily_ret.iloc[0]=daily_ret.iloc[1]
daily_cov=daily_ret.cov() #공분산 처음써봄
annual_ret=daily_ret.mean()*250 #annual값이 필요했던 것
annual_cov=daily_cov*250 
port_ret=[] 
port_risk=[]
port_weights=[]
port_sharpe=[]

In [ ]:
import numpy as np
for _ in range(20000):
  weights=np.random.random(4) #np패키지속 random모듈속 random함수
  weights=weights/np.sum(weights) #난수가 범위가 없으니깐
  returns=np.dot(weights,annual_ret) 
  risk=np.sqrt(np.dot(weights.T,np.dot(annual_cov,weights))) #포트폴리오 리스크공식(이론상과 약간다른가?) 
  sharpe=(returns-0.005)/risk #기준금리적용
  port_ret.append(returns)
  port_risk.append(risk)
  port_weights.append(weights)
  port_sharpe.append(sharpe)
portfolio={'Return':port_ret,'Risk':port_risk,'Sharpe':port_sharpe} #딕셔너리는 굉장히 애용되는 자료형이구나
stocks=['셀트리온','LG전자','삼성전자','카카오']
for i,s in enumerate(stocks): #인덱스가 앞임
  portfolio[s]=[weight[i] for weight in port_weights] #딕셔너리 입력을 이렇게도 가능하구나 #각 리스트의 가중치마다
port_df=pd.DataFrame(portfolio)

In [ ]:
plt.scatter(data=port_df,x='Risk',y='Return')
plt.title('Efficient portfolio')
plt.xlabel('Risk')
plt.ylabel('Expected Return')
max_sharpe=port_df.loc[port_df['Sharpe']==port_df['Sharpe'].max()] #마킹하는 법(dataframe위 한 점 선택)
plt.scatter(x=max_sharpe['Risk'],y=max_sharpe['Return'],c='r',marker='x',s=300) #사이즈 지정필수
plt.show()

In [ ]:
max_sharpe

#3.매수,매도 시점

##1)종목1

###• 볼린저 밴드

In [ ]:
df['MA20']=df['Close'].rolling(window=20,min_periods=1).mean() #볼린저밴드자체는 MA20으로 만든다
std=df['Close'].rolling(window=20,min_periods=1).std() #둘다 close rolling해서 만든다
std.fillna(method='bfill',inplace=True) #std만 초깃값이 빈다
df['Upper']=df['MA20']+2*std
df['Lower']=df['MA20']-2*std
df['%B']=100*(df.Close-df.Lower)/(df.Upper-df.Lower)
df['TP']=(df['High']+df['Lower']+df['Close'])/3 #MFI만 필요한 수치
df['PMF']=0
df['NMF']=0
df

In [ ]:
for i in range(len(df)-1): #0부터 개수-2까지
  if df.TP.values[i]<df.TP.values[i+1]: 
    df.PMF.values[i+1]=df.TP.values[i+1]*df.Volume.values[i+1] #그냥 값자체에 가중치곱하면 흐름으로 처리
    df.NMF.values[i+1]=0
  elif df.TP.values[i]>df.TP.values[i+1]:
    df.NMF.values[i+1]=df.TP.values[i+1]*df.Volume.values[i+1] #그냥 값자체에 가중치곱하면 흐름으로 처리
    df.PMF.values[i+1]=0
MF=df.PMF.rolling(window=10).sum()/df.NMF.rolling(window=10).sum() #유일하게 10으로 rolling
df['MFI']=100-100/(1+MF)

In [ ]:
plt.subplot(2,1,1)
plt.plot(df.index,df.Close,label='Close')
plt.plot(df.index,df.Upper,label='Upper')
plt.plot(df.index,df.Lower,label='Lower')
plt.plot(df.index,df.MA20,label='MA20')
plt.legend(loc='best')
plt.subplot(2,1,2)
plt.plot(df.index,df['%B'],label='%B')
plt.plot(df.index,df.MFI,label='MFI')
buy=[]
sell=[]
df.index=df.index.date #날짜만!
for i in range(len(df)):
  if df['%B'].values[i]>80 and df.MFI.values[i]>80:
    plt.plot(df.index.values[i],0,'r^')
    buy.append(df.index.values[i])
  if df['%B'].values[i]<20 and df.MFI.values[i]<20:
    plt.plot(df.index.values[i],0,'bv')
    sell.append(df.index.values[i])
plt.legend(loc='best')

In [ ]:
intensity=df.Volume*(2*df.Close-df.High-df.Low)/(df.High-df.Low) #거래량을 곱해준다음 rolling해서 나눠주기
df['%I']=100*intensity.rolling(window=20).sum()/df.Volume.rolling(window=20).sum()
plt.bar(df.index,df['%I'],label='%I in 20days')

###• 삼중창

In [ ]:
L14=df.Low.rolling(window=14,min_periods=1).min() 
H14=df.High.rolling(window=14,min_periods=1).max()
df['%K']=100*(df['Close']-L14)/(H14-L14)
df['%D']=df['%K'].rolling(window=3).mean() 
plt.plot(df.index,df['%D'],label='%D')
plt.title('2nd Screen')
plt.legend(loc='best')

In [ ]:
df['EMA130']=df['Close'].ewm(span=130).mean()
plt.plot(df.index,df['EMA130'],label='EMA130')
for i in range(len(df)-1):
  if df.EMA130.values[i]<df.EMA130.values[i+1] and df['%D'].values[i]<=20 and df['%D'].values[i+1]>20:
    plt.plot(df.index[i+1],df.EMA130.mean(),'r^') #평균으로 해야 마커가 일관성 있게 보임
    buy.append(df.index.values[i])
  elif df.EMA130.values[i]>=df.EMA130.values[i+1] and df['%D'].values[i]>=80 and df['%D'].values[i+1]<80: 
    plt.plot(df.index[i+1],df.EMA130.mean(),'bv')
    sell.append(df.index.values[i])
plt.title('3rd Screen')

###• 종합

In [ ]:
print('매수시점:',buy)
print('매도시점:',sell)

##2)종목2

###• 볼린저 밴드

###• 삼중창

###• 종합

##3)종목3

###• 볼린저 밴드

###• 삼중창

###• 종합

#4.예측


##1)종목1

In [ ]:
scaled_df=MinMaxScaler().fit_transform(k_df) #list반환 주의!
scaled_df=pd.DataFrame(scaled_df,columns=['Open','High','Low','Close','Volume'])
target_df=scaled_df['Close'] #target도 scale ver

In [ ]:
x=scaled_df.values.tolist() #텐서플로의 신기함(dataframe에서 ndarray에서 list로)
y=target_df.values.tolist()
xdata=[]
ydata=[]
window=10 #10일씩 묶어서 훨씬 늘림
for i in range(len(y)-window):
  x_=x[i:i+window] #9개씩
  y_=y[i+window] #종가는 하나씩
  xdata.append(x_)
  ydata.append(y_)
print(x_,"->",y_) #맨마지막

In [ ]:
test_size=int(len(ydata)*0.3) #공부를 좀 더 해서 순서대로 하는게 맞는 건지등 ROCV는 뭔지등 공부(아직 멀었다!)
X_train=np.array(xdata[0:len(xdata)-test_size+1]) #개수 맞추기
X_test=np.array(xdata[len(xdata)-test_size:len(xdata)])
y_train=np.array(ydata[0:len(ydata)-test_size+1])
y_test=np.array(ydata[len(ydata)-test_size:len(ydata)])

In [ ]:
model=Sequential()
model.add(LSTM(units=10,activation='relu',input_shape=(window,5),return_sequences=True)) #마지막에 과정 표현여부 선택
model.add(Dropout(0.1))
model.add(LSTM(units=10,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=1))
model.summary() #layer구조확인

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error') #딥러닝은 compile후 fit_predict
model.fit(X_train,y_train,epochs=60,batch_size=30) #같은 데이터로 몇번 반복/몇건씩 입력할건지
pred=model.predict(X_test)

In [ ]:
plt.plot(y_test,color='red',label='real')
plt.plot(pred,color='blue',label='prediction')
plt.title('Real VS Prediction')
plt.xlabel('time')
plt.ylabel('price')
plt.legend(loc='best')
plt.show()

In [ ]:
print("내일 카카오톡 주가는 {0}원".format(pred[-1]*k_df.Close.iloc[-1]/scaled_df.Close.iloc[-1])) #다시 scale돌려놓기

##2)종목2

##3)종목3

#5.결론